In [ ]:
import numpy as np # for arrays, Series, List, Sets
import pandas as pd # for dataframes

#- 9 meses: 26483
#- 9 meses without package: 26006 
#- 9 meses without cobranding: 23646 
#- todo junto: 23191

In [ ]:
data = pd.read_csv('data.csv', sep='|', skipfooter=1, engine='python')

# Universo de clientes

In [ ]:
data.shape

In [ ]:
data['client_id'].nunique()

In [ ]:
sorted(data['Month'].unique())

In [ ]:
client_month_counts = data.groupby('client_id')['Month'].nunique()
print(client_month_counts)

In [ ]:
clients_with_9_months = client_month_counts[client_month_counts == 9].index
print(clients_with_9_months.shape)

In [ ]:
data2 = data[data['client_id'].isin(clients_with_9_months)]

data2['client_id'].nunique()
# data2: Dataframe con solo clientes con 9 meses

In [ ]:
data2.head()

In [ ]:
data2.shape

In [ ]:
data2['CreditCard_CoBranding'].value_counts().reset_index()

In [ ]:
# 6 meses training, 1 mes lead, 2 meses prediction
mask = (data2['Month'] == '2019-01-01') & (data2['CreditCard_CoBranding'] != 'No')

clients_to_remove = data2.loc[mask, 'client_id'].unique()

data3 = data2[~data2['client_id'].isin(clients_to_remove)]

data3['client_id'].nunique()
# data3: Dataframe con clientes sin cobranding en el mes 6. Solo clientes con 9 meses.

In [ ]:
data3.shape

In [ ]:
mask2 = (data3['Month'] == '2019-01-01') & (data3['Package_Active'] != 'No')

clients_to_remove2 = data3.loc[mask2, 'client_id'].unique()

data4 = data3[~data3['client_id'].isin(clients_to_remove2)]

data4['client_id'].nunique()
# data4: clientes sin paquete ni cobranding en el mes 6. Solo clientes con 9 meses.

# Tratamiento de nulos

In [ ]:
data4['Region'].value_counts(dropna=False).reset_index()
# detectados 162349 nulos en Region:

In [ ]:
data4.head()

# Algunos Identity Features:
# client_id, Month, Sex

# Algunos Transform Featues:
# Client_Age_grp, First_product_dt, Last_product_dt

In [ ]:
data5 = data4.copy()

data_only_one = data5[(data5['client_id'] == 5856970)]
data_only_one[['client_id', 'Region', 'CreditCard_Product', 'Month']]

# se ven muchos clientes con region y creditProduct solo en los meses 2019-03 y 2019-04:

In [ ]:
data_region = data5[(data5['Month'] == '2019-04-01')]
data_region[['Region', 'Month']].value_counts(dropna=False).reset_index()

# existen 6 clientes que no tienen Region para el mes 2019-04:

In [ ]:
result_clients = data5[(data5['Month'] == '2019-04-01') & (data5['Region'].isna())]['client_id']
result_clients
# clientes sin regiones registradas en el mes 2019-04

In [ ]:
# se transfieren las 2 regiones repetidas de cada cliente, a todo el resto de sus meses
# para eliminar la mayoria de nulos en Region

client_region_map = data5[data5['Month'] == '2019-04-01'].set_index('client_id')['Region'].to_dict()

data5['Region'] = data5['client_id'].map(client_region_map)


data_only_one = data5[(data5['client_id'] == 5856970)]
data_only_one[['client_id', 'Region', 'CreditCard_Product', 'Month']]

In [ ]:
# se inserta Desconocido en los 6 clientes con region desconocida
# para eliminar los últimos nulos en Region

data5.loc[data5['client_id'].isin(result_clients), 'Region'] = 'Desconocido'

data_region2 = data5[(data5['Month'] == '2019-04-01')]
data_region2[['Region', 'Month']].value_counts(dropna=False).reset_index()

In [ ]:
data5['Region'].value_counts(dropna=False).reset_index()

# en resumen, paso de 162349 NaN, a 54 desconocidos

In [ ]:
# data5: Dataframe sin nulos en Region
data5[['CreditCard_Product']].value_counts(dropna=False).reset_index()
# detectados 178520 nulos en CreditCard_Product:

In [ ]:
data_credit = data5[(data5['Month'] == '2019-04-01')]
data_credit[['CreditCard_Product', 'Month']].value_counts(dropna=False).reset_index()
# existen 8102 clientes que no tienen CreditCard_Product para el mes 2019-04:

In [ ]:
result_clients2 = data5[(data5['Month'] == '2019-04-01') & (data5['CreditCard_Product'].isna())]['client_id']
result_clients2
# clientes sin credit_product registradas en el mes 2019-04:

In [ ]:
# se transfieren las 2 CreditCard_Product repetidos en 2019-03 y 2019-04 de cada cliente, a todo el resto de sus meses

data6 = data5.copy()

client_credit_map = data6[data6['Month'] == '2019-04-01'].set_index('client_id')['CreditCard_Product'].to_dict()

data6['CreditCard_Product'] = data6['client_id'].map(client_credit_map)


data_only_one2 = data6[(data6['client_id'] == 5856970)]
data_only_one2[['client_id', 'Region', 'CreditCard_Product', 'Month']]

In [ ]:
# se inserta Desconocido en los 8102 clientes con CreditCard_Product desconocido

data6.loc[data6['client_id'].isin(result_clients2), 'CreditCard_Product'] = 'Desconocido'

data_credit2 = data6[(data6['Month'] == '2019-04-01')]
data_credit2[['CreditCard_Product', 'Month']].value_counts(dropna=False).reset_index()

In [ ]:
data6[['CreditCard_Product']].value_counts(dropna=False).reset_index()

# en resumen, paso de 178520 NaN, a 72918 desconocidos

# Outliers

In [ ]:
# data6: Dataframe sin nulos en Region ni CreditCard_Product

# Checkeo de outliers
pd.set_option('display.max_columns', None)
data6.head()

In [ ]:
data6[['SavingAccount_Balance_FirstDate']].value_counts(dropna=False).reset_index()
# Ejemplo de outliers en SavingAccount_Balance_FirstDate

In [ ]:
data6_ordered = data6.sort_values('SavingAccount_Balance_FirstDate')
data6_ordered[['SavingAccount_Balance_FirstDate']]

In [ ]:
data6['SavingAccount_Balance_FirstDate'].value_counts().plot(kind='box')

In [ ]:
data6['SavingAccount_Balance_FirstDate'].describe().apply(lambda x: format(x, 'f'))

In [ ]:
[x for x in data6.columns if (x.startswith('SavingAccount_') or x.startswith('Operations_') or x.startswith('CreditCard_'))]

In [ ]:
data6['CreditCard_Total_Limit'].value_counts(dropna=False).reset_index()#describe().apply(lambda x: format(x, 'f'))

#### muchos valores
CreditCard_Revolving
CreditCard_Spending_Aut_Debits
CreditCard_Spending_CrossBoarder
CreditCard_Spending_Installments
CreditCard_Spending_1_Installment
CreditCard_Total_Spending
CreditCard_Balance_DOLLAR
CreditCard_Balance_ARG
SavingAccount_Balance_Average
SavingAccount_Balance_FirstDate
SavingAccount_Balance_LastDate
SavingAccount_Salary_Payment_Amount
SavingAccount_Transfer_In_Amount
SavingAccount_ATM_Extraction_Amount
SavingAccount_Service_Payment_Amount
SavingAccount_CreditCard_Payment_Amount
SavingAccount_Transfer_Out_Amount
SavingAccount_DebitCard_Spend_Amount
SavingAccount_Total_Amount
SavingAccount_Credits_Amounts
SavingAccount_Debits_Amounts

#### valores concretos grandes
CreditCard_Total_Limit

In [ ]:
data6['CreditCard_Total_Limit'].describe()#.apply(lambda x: format(x, 'f'))

In [ ]:
columnas_con_outliers = [col for col in data6.columns if (data6[col].nunique() > 200) and (data6[col].dtype == 'float64')]
print(columnas_con_outliers)
# Detección de columnas con outliers de forma automática

In [ ]:
print(len(columnas_con_outliers))

In [ ]:
# data6[data6['client_id'] == 5856970][['client_id', 'Month', 'SavingAccount_Balance_Average']].reset_index()

data6['SavingAccount_Balance_Average'].describe().apply(lambda x: format(x, 'f'))

In [ ]:
# ejemplo de borrar outliers
p95 = data6[data6.SavingAccount_Balance_Average > 120.0]['SavingAccount_Balance_Average'].quantile(0.95)
p99 = data6[data6.SavingAccount_Balance_Average > 120.0]['SavingAccount_Balance_Average'].quantile(0.99)
three_sigma = 3 * data6[data6.SavingAccount_Balance_Average > 139.4]['SavingAccount_Balance_Average'].std()

print('p95		', p95)
print('p99		', p99)
print('three_sigma	', three_sigma)

In [ ]:
sin_outlier = np.where(data6['SavingAccount_Balance_Average'] > three_sigma, three_sigma, data6['SavingAccount_Balance_Average'])
sin_outlier_df = pd.DataFrame(sin_outlier, columns=['SavingAccount_Balance_Average'])
sin_outlier_df.describe()

In [ ]:
data7 = data6.copy()
for column in columnas_con_outliers:
    three_sigma = 3 * data6[data6[column] > 1][column].std()
    data7[column] = np.where(data6[column] > three_sigma, three_sigma, data6[column])

# data7: nuevo dataframe sin outliers en las 21 columnas detectadas automáticamente

In [ ]:
data7['SavingAccount_Balance_Average'].describe().apply(lambda x: format(x, 'f'))

data7.head()

##### Identity features, columnas para pasar directamente:
(todas en el ultimo mes de training)
client_id
Sex
Region
Mobile
Email
CreditCard_Product


##### Transform features, agrupamientos, o cantidad de algo
numero de insurance y active bools en yes. Opcional convertir a dtype bool para Numero de bools totales en yes
Client_Age_grp, pasar a numero
diferencia en dias de First_product_dt - Last_product_dt. Diferencia del ultimo con ultimo mes de training

##### Aggregate features
operaciones, sum avg min max etc, para 6 y para 3 meses, generadas automaticamente, para todas las numericas

# Tratamiento de categóricas

In [ ]:
data8 = data7.copy()
sorted(data7['Client_Age_grp'].unique())

In [ ]:
# Integer encoding para edades
age_mapping = {
    'Menor a 18 años': 0,
    'Entre 18 y 29 años': 1,
    'Entre 30 y 39 años': 2,
    'Entre 40 y 49 años': 3,
    'Entre 50 y 59 años': 4,
    'Entre 60 y 64 años': 5,
    'Entre 65 y 69 años': 6,
    'Mayor a 70 años': 7
}
# data8.loc[:, 'Client_Age_grp'] = data8['Client_Age_grp'].map(age_mapping)
data8['Client_Age_grp'] = data8['Client_Age_grp'].map(age_mapping)

In [ ]:
# Columnas bool a numero
bool_columns = ["Mobile", "Email", "CreditCard_Premium", "CreditCard_Active", "CreditCard_CoBranding", "Loan_Active", "Mortgage_Active", "SavingAccount_Active_ARG_Salary", "SavingAccount_Active_ARG", "SavingAccount_Active_DOLLAR", "DebitCard_Active", "Investment_Active", "Package_Active", "Insurance_Life", "Insurance_Home", "Insurance_Accidents", "Insurance_Mobile", "Insurance_ATM", "Insurance_Unemployment"]
bool_mapping = {
    'No': 0,
    'Yes': 1
}
data8[bool_columns] = data8[bool_columns].replace(bool_mapping)

In [ ]:
# Sexo
sex_mapping = {
    'F': 0,
    'M': 1
}
data8['Sex'] = data8['Sex'].map(sex_mapping)

In [ ]:
# Faltan mapear: 
["Region", "CreditCard_Product", "First_product_dt", "Last_product_dt"]
# Pero requieren columnas nuevas

In [ ]:
data8.head()
# data8: dataframe sin columas de texto

# Aggregate features por categoria

In [ ]:
# Lista de categorias:
columns_savingAccount = [x for x in data8.columns if x.startswith('SavingAccount_')]
columns_operations = [x for x in data8.columns if x.startswith('Operations_')]
columns_creditCard = [x for x in data8.columns if x.startswith('CreditCard_') and x != 'CreditCard_Product']
columns_active = [x for x in data8.columns if 'Active' in x]
columns_insurance = [x for x in data8.columns if x.startswith('Insurance_')]

# incluir tambien cada columna sin transformar

# Columnas sin aggregated:
# 	First_product_dt	Last_product_dt	Sex	Client_Age_grp	Investment_Numbers	Mobile	Email	Region	CreditCard_Product

In [ ]:
# calcular aggregate features

seis_meses = ['2018-08-01',
             '2018-09-01',
             '2018-10-01',
             '2018-11-01',
             '2018-12-01',
             '2019-01-01']
tres_meses = ['2018-11-01',
             '2018-12-01',
             '2019-01-01']

seis_data = data8[data8['Month'].isin(seis_meses)]

# columnas_agg = [col for col in data7.columns if ((data7[col].dtype == 'float64') and (col not in (['Target', 'client_id'])))]

# np.sum, np.amax, np.min, np.mean, np.median, np.count_nonzero, 'nunique', np.var
agg_feat_savingAccount = seis_data.groupby(['client_id'])[columns_savingAccount].agg([np.sum, np.amax, np.min, np.mean, np.median, np.count_nonzero, 'nunique', np.var])
agg_feat_operations = seis_data.groupby(['client_id'])[columns_operations].agg([np.sum, np.amax, np.min, np.mean, np.median, np.count_nonzero, 'nunique', np.var])
agg_feat_creditCard = seis_data.groupby(['client_id'])[columns_creditCard].agg([np.sum, np.amax, np.min, np.mean, np.median, np.count_nonzero, 'nunique', np.var])
agg_feat_active = seis_data.groupby(['client_id'])[columns_active].agg([np.sum, np.amax, np.min, np.mean, np.median, np.count_nonzero, 'nunique', np.var])
agg_feat_insurance = seis_data.groupby(['client_id'])[columns_insurance].agg([np.sum, np.amax, np.min, np.mean, np.median, np.count_nonzero, 'nunique', np.var])


agg_feat_savingAccount.columns = ['_seis_'.join(x) for x in np.array(agg_feat_savingAccount.columns)]
agg_feat_operations.columns = ['_seis_'.join(x) for x in np.array(agg_feat_operations.columns)]
agg_feat_creditCard.columns = ['_seis_'.join(x) for x in np.array(agg_feat_creditCard.columns)]
agg_feat_active.columns = ['_seis_'.join(x) for x in np.array(agg_feat_active.columns)]
agg_feat_insurance.columns = ['_seis_'.join(x) for x in np.array(agg_feat_insurance.columns)]


### Remover  correlacionadas

In [ ]:
agg_feat_savingAccount.head()

In [ ]:
print(agg_feat_savingAccount.shape)

In [ ]:
def remove_highly_correlated_features(agg_feat):

    agg_feat_corr = agg_feat.reset_index().copy()
    # Create correlation matrix

    corr_matrix = agg_feat_corr.drop('client_id', axis=1).corr().abs()

    # Select upper triangle of correlation matrix

    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)).fillna(0)

    # Find features with correlation greater than 0.80

    to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]

    #upper.reset_index().to_csv('agg_feat_corr.csv', sep='|',  header=True , encoding='ANSI', index=False)

    # print(to_drop)

    # Drop features 

    agg_feat_corr.drop(to_drop, axis=1, inplace = True)

    agg_feat_corr.shape

    return agg_feat_corr

In [ ]:
sin_corr_savingAccount = remove_highly_correlated_features(agg_feat_savingAccount)
sin_corr_savingAccount.shape

In [ ]:
sin_corr_savingAccount.head()

In [ ]:
print('Antes operations: ')
print(agg_feat_operations.shape)
print('Antes creditCard: ')
print(agg_feat_creditCard.shape)
print('Antes active: ')
print(agg_feat_active.shape)
print('Antes insurance: ')
print(agg_feat_insurance.shape)
print('\n')

sin_corr_operations = remove_highly_correlated_features(agg_feat_operations)
sin_corr_creditCard = remove_highly_correlated_features(agg_feat_creditCard)
sin_corr_active = remove_highly_correlated_features(agg_feat_active)
sin_corr_insurance = remove_highly_correlated_features(agg_feat_insurance)

print('Despues operations: ')
print(sin_corr_operations.shape)
print('Despues creditCard: ')
print(sin_corr_creditCard.shape)
print('Despues active: ')
print(sin_corr_active.shape)
print('Despues insurance: ')
print(sin_corr_insurance.shape)

### Scaler

In [ ]:
# correr scaler
from sklearn.preprocessing import StandardScaler

def scale_dataframe(dataframe):

    model = dataframe.copy()
    names = [x for x in model.columns if (x != 'client_id') & (x != 'Target')]

    scaler = StandardScaler(copy=True)
    scaler.fit(model[names])
    scaled_est = scaler.transform(model[names])
    scaled_est = pd.DataFrame(scaled_est, columns=names, index=model.index)

    model.drop(names, axis=1, inplace=True)
    model = pd.concat((model, scaled_est), axis=1, sort=False)
    
    return model

In [ ]:
scaled_savingAccount = scale_dataframe(sin_corr_savingAccount)
scaled_operations = scale_dataframe(sin_corr_operations)
scaled_creditCard = scale_dataframe(sin_corr_creditCard)
scaled_active = scale_dataframe(sin_corr_active)
scaled_insurance = scale_dataframe(sin_corr_insurance)

scaled_savingAccount.head()

# Creacion de ABT por categoria

In [ ]:
ultimomes = '2019-01-01'
data_ultimomes = data8[data8['Month'] == ultimomes].copy()


In [ ]:

ABT_savingAccount = pd.merge(data_ultimomes[['client_id', 'Target']], scaled_savingAccount, on='client_id', how='inner')
ABT_operations = pd.merge(data_ultimomes[['client_id', 'Target']], scaled_operations, on='client_id', how='inner')
ABT_creditCard = pd.merge(data_ultimomes[['client_id', 'Target']], scaled_creditCard, on='client_id', how='inner')
ABT_active = pd.merge(data_ultimomes[['client_id', 'Target']], scaled_active, on='client_id', how='inner')
ABT_insurance = pd.merge(data_ultimomes[['client_id', 'Target']], scaled_insurance, on='client_id', how='inner')

ABT_demographics = data_ultimomes[['client_id', 'Target', 'Sex', 'Client_Age_grp', 'Mobile', 'Email', 'Investment_Numbers']]
# First_product_dt	Last_product_dt		Region	CreditCard_Product

In [ ]:
ABT_insurance.head()

### ABT_demographics

In [ ]:
ABT_demographics.head()

In [ ]:
# carga de fechas en número
from datetime import datetime

data_ultimomes['Last_product_dt'] = pd.to_datetime(data_ultimomes['Last_product_dt'])
data_ultimomes['First_product_dt'] = pd.to_datetime(data_ultimomes['First_product_dt'])

data_ultimomes['days_last_first'] = (data_ultimomes['Last_product_dt'] - data_ultimomes['First_product_dt']).dt.days

today_date = datetime.strptime(ultimomes, '%Y-%m-%d')
data_ultimomes['days_today_last'] = (today_date - data_ultimomes['Last_product_dt']).dt.days
data_ultimomes['days_today_last'] = data_ultimomes['days_today_last'].apply(lambda x: 0 if x < 0 else x)


ABT_demographics = pd.merge(ABT_demographics, data_ultimomes[['client_id', 'days_last_first', 'days_today_last']], on='client_id', how='inner')

ABT_demographics.head()


In [ ]:
data_ultimomes['Region'].value_counts()

In [ ]:

def target_print(df, columnName):
    return df.groupby(columnName).agg(Clientes=('Target', 'count'), Target_1=('Target', lambda x: (x == 1).sum()), Percentage_Target=('Target', lambda x: (x == 1).sum() / x.count() * 100)).reset_index().sort_values(by='Percentage_Target', ascending=False)

In [ ]:
target_print(data_ultimomes, 'Region')

In [ ]:
target_print(data_ultimomes, 'CreditCard_Product')

In [ ]:
# Borrar rows para los onehot

dummy_data = data_ultimomes[~data_ultimomes['CreditCard_Product'].isin(['Desconocido', 'J55660123XX012'])]
dummy_data = dummy_data[~dummy_data['Region'].isin(['Desconocido'])]

In [ ]:
target_print(dummy_data, 'Region')

In [ ]:
target_print(dummy_data, 'CreditCard_Product')

In [ ]:
# Las rows borradas resultaron afectar a multiples columnas, asi que no lo considero por ahora
# Agrego una columa por cada una, porque ninguna forma de agruparlas me parece equitativa, lamentablemente quedarian un par con pocos registros

In [ ]:
ABT_demographics2 = pd.get_dummies(data_ultimomes, prefix_sep="_dummy_", columns=['Region', 'CreditCard_Product'], drop_first=True)

In [ ]:
columns_dummy = [x for x in ABT_demographics2.columns if 'dummy' in x]
columns_dummy = np.append(columns_dummy, 'client_id')
ABT_demographics2.head()


In [ ]:
ABT_demographics = pd.merge(ABT_demographics, ABT_demographics2[columns_dummy], on='client_id', how='inner')

In [ ]:
ABT_demographics.head()

In [ ]:
list_ABT = ['ABT_savingAccount', 'ABT_operations', 'ABT_creditCard', 'ABT_active', 'ABT_insurance', 'ABT_demographics']
list_ABT

# Evaluar mejores variables por categoria

In [ ]:
# evaluar mejores variables

# conda install -c conda-forge lightgbm
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

target_column = 'Target'

In [ ]:
def evaluar_variables(ABT_model, numerical_cols, target_column):

    fit_params={#"early_stopping_rounds":100,
               "eval_metric": 'auc',
               "eval_set": [(ABT_model[numerical_cols], ABT_model[target_column])]
                #,"verbose": 100
               }
    param_test = {'num_leaves': np.arange(5, 20, 1),
                 'min_data_in_leaf': np.arange(10, 100, 1),
                 'subsample': sp_uniform(loc=0.2, scale=0.8),
                 'max_depth': np.arange(5, 20, 1),
                 'n_estimators': np.arange(20, 100, 1)
                 }
    n_HP_points_to_test = 100


    clf = lgb.LGBMClassifier(random_state=314, silent=True, metric='None', nfold=5, njobs=4, early_stopping_rounds=100)

    gs = RandomizedSearchCV(estimator=clf, param_distributions=param_test,
                           n_iter=n_HP_points_to_test,
                           scoring='roc_auc',
                           cv=3,
                           refit=True,
                           random_state=314,
                           verbose=True,
                           error_score='raise')

    gs.fit(ABT_model[numerical_cols], ABT_model[target_column], **fit_params)
    #feat_imp = pd.Series(gs.best_estimator_.feature_importances, index=ABT_model[numerical_cols].columns)
    feat_imp = pd.Series(gs.best_estimator_.booster_.feature_importance(importance_type='gain'), index=ABT_model[numerical_cols].columns)

    return gs, feat_imp


In [ ]:

cols_savingAccount = [x for x in ABT_savingAccount.columns if (x != 'client_id') & (x != target_column)]
gs_savingAccount, feat_savingAccount = evaluar_variables(ABT_savingAccount, cols_savingAccount, target_column)


In [ ]:
print(gs_savingAccount.best_estimator_.get_params())

In [ ]:
feat_savingAccount.nlargest(20).plot(kind='barh', figsize=(8, 10))

In [ ]:
feat_savingAccount.nlargest(5)

In [ ]:
top_savingAccount = feat_savingAccount.nlargest(5).index.tolist()

In [ ]:

cols_operations = [x for x in ABT_operations.columns if (x != 'client_id') & (x != target_column)]
gs_operations, feat_operations = evaluar_variables(ABT_operations, cols_operations, target_column)


In [ ]:
print(gs_operations.best_estimator_.get_params())

In [ ]:

feat_operations.nlargest(20).plot(kind='barh', figsize=(8, 10))

In [ ]:
top_operations = feat_operations.nlargest(5).index.tolist()
feat_operations.nlargest(5)

In [ ]:

cols_creditCard = [x for x in ABT_creditCard.columns if (x != 'client_id') & (x != target_column)]
gs_creditCard, feat_creditCard = evaluar_variables(ABT_creditCard, cols_creditCard, target_column)

In [ ]:
print(gs_creditCard.best_estimator_.get_params())

In [ ]:
feat_creditCard.nlargest(20).plot(kind='barh', figsize=(8, 10))

In [ ]:
top_creditCard = feat_creditCard.nlargest(5).index.tolist()
feat_creditCard.nlargest(5)

In [ ]:

cols_active = [x for x in ABT_active.columns if (x != 'client_id') & (x != target_column)]
gs_active, feat_active = evaluar_variables(ABT_active, cols_active, target_column)


In [ ]:
print(gs_active.best_estimator_.get_params())

In [ ]:
feat_active.nlargest(20).plot(kind='barh', figsize=(8, 10))

In [ ]:
top_active = feat_active.nlargest(3).index.tolist()
feat_active.nlargest(3)

In [ ]:

cols_insurance = [x for x in ABT_insurance.columns if (x != 'client_id') & (x != target_column)]
gs_insurance, feat_insurance = evaluar_variables(ABT_insurance, cols_insurance, target_column)

In [ ]:
print(gs_insurance.best_estimator_.get_params())

In [ ]:
feat_insurance.nlargest(20).plot(kind='barh', figsize=(8, 10))

In [ ]:
top_insurance = feat_insurance.nlargest(3).index.tolist()
feat_insurance.nlargest(3)

In [ ]:

cols_demographics = [x for x in ABT_demographics.columns if (x != 'client_id') & (x != target_column)]
gs_demographics, feat_demographics = evaluar_variables(ABT_demographics, cols_demographics, target_column)

In [ ]:
print(gs_demographics.best_estimator_.get_params())

In [ ]:
feat_demographics.nlargest(20).plot(kind='barh', figsize=(8, 10))


In [ ]:
top_demographics = feat_demographics.nlargest(5).index.tolist()
feat_demographics.nlargest(5)

## Variables seleccionadas

In [ ]:
print(top_savingAccount)
print(top_operations)
print(top_creditCard)
print(top_active)
print(top_insurance)
print(top_demographics)

In [ ]:

ABT_final = data_ultimomes[['client_id', 'Target']]


['ABT_savingAccount',
 'ABT_operations',
 'ABT_creditCard',
 'ABT_active',
 'ABT_insurance',
 'ABT_demographics']

ABT_final = pd.merge(ABT_final, ABT_savingAccount[np.append(top_savingAccount, 'client_id')], on='client_id', how='inner')
ABT_final = pd.merge(ABT_final, ABT_operations[np.append(top_operations, 'client_id')], on='client_id', how='inner')
ABT_final = pd.merge(ABT_final, ABT_creditCard[np.append(top_creditCard, 'client_id')], on='client_id', how='inner')
ABT_final = pd.merge(ABT_final, ABT_active[np.append(top_active, 'client_id')], on='client_id', how='inner')
ABT_final = pd.merge(ABT_final, ABT_insurance[np.append(top_insurance, 'client_id')], on='client_id', how='inner')
ABT_final = pd.merge(ABT_final, ABT_demographics[np.append(top_demographics, 'client_id')], on='client_id', how='inner')
ABT_final.head()

In [ ]:
ABT_final.shape

# Bivariado

In [ ]:
# Rename para bivariado
ABT = ABT_final.copy()
ABT.rename(columns={'client_id': 'idx'}, inplace=True)
ABT.rename(columns={'Target': 'TGT'}, inplace=True)
ABT.head()

In [ ]:
# Bivariados de las 2 top variables obtenidas luego

%run ./Graficos_v2.py

vars = ['idx', 'TGT', 'SavingAccount_CreditCard_Payment_Amount_seis_sum']

Graficar_Variables2(ABT[vars], [], 'TGT')

In [ ]:
%run ./Graficos_v2.py

vars = ['idx', 'TGT', 'CreditCard_Product_dummy_J55660104XX012']

Graficar_Variables2(ABT[vars], [], 'TGT')

# Split in train and Test

In [ ]:
top_variables = [col for col in ABT_final.columns if (col not in (['Target', 'client_id', 'CreditCard_Active_seis_sum_x', 'CreditCard_Active_seis_sum_y', 'CreditCard_Total_Spending_seis_sum']))]
top_variables

# Borro 'CreditCard_Active_seis_sum', 'CreditCard_Total_Spending_seis_sum'
# Para que no quede una sola variable ganadora

In [ ]:

# Split in train and Test
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(ABT_final, test_size=0.3, random_state=420, stratify=ABT_final['Target'])

print(x_train.Target.value_counts())
print(x_test.Target.value_counts())

In [ ]:
x_train.head()

In [ ]:
# Balancear dataset de training
minority_class_count = x_train['Target'].value_counts().min()

undersampled_majority_class = x_train.groupby('Target').apply(lambda x: x.sample(minority_class_count)).reset_index(drop=True)

print(undersampled_majority_class['Target'].value_counts())

In [ ]:
x_train = undersampled_majority_class
print(x_train.Target.value_counts())

# Entrenar

In [ ]:
# Entrenar

fit_params2={#"early_stopping_rounds":100,
           "eval_metric": 'auc',
           "eval_set": [(x_test[top_variables], x_test[target_column])]
            #,"verbose": 100
           }
param_test2 = {'num_leaves': np.arange(5, 10, 1),
             'min_child_samples': np.arange(300, 1000, 100),
             'learning_rate': [0.05,0.1,0.2,0.3,0.4],
             'max_depth': np.arange(1, 7, 1),
             'n_estimators': np.arange(12, 24, 1)
             }
n_HP_points_to_test = 100


clf2 = lgb.LGBMClassifier(random_state=314, silent=True, metric='None', njobs=4, early_stopping_rounds=100)

gs2 = RandomizedSearchCV(estimator=clf2, param_distributions=param_test2,
                       n_iter=n_HP_points_to_test,
                       scoring='roc_auc',
                       cv=3,
                       refit=True,
                       random_state=314,
                       verbose=True,
                       error_score='raise')

gs2.fit(x_train[top_variables], x_train[target_column], **fit_params2)
#feat_imp2 = pd.Series(gs2.best_estimator_.feature_importances, index=ABT_model[top_variables].columns)
feat_imp2 = pd.Series(gs2.best_estimator_.booster_.feature_importance(importance_type='gain'), index=x_train[top_variables].columns)



In [ ]:
feat_imp2.nlargest(20).plot(kind='barh', figsize=(8, 10))

In [ ]:
feat_imp2.nlargest(20)

In [ ]:
print(gs2.best_estimator_.get_params())

# Extraer deciles

In [ ]:
# Scoreo la base completa con el modelo final (mejores hiperparametros)
clf_final_train = gs2


from sklearn.metrics import mean_squared_error

probabilities_train = clf_final_train.predict_proba(x_train[top_variables])
probabilities       = clf_final_train.predict_proba(x_test[top_variables])


# Con esto obtenes los deciles, cantidad de personas, buenas y total, junto a la cota de probabilidad por decil para training

a = pd.DataFrame(x_train[['client_id', target_column]], columns=['Target', 'client_id'])
a = a.reset_index()
b = pd.DataFrame(probabilities_train[:,1], columns=['Prob1'])

result = pd.concat([a, b], axis=1)
result['porc'] = result['Prob1'].rank(pct=True) * 100
len(probabilities[:,1])

result.loc[result['porc'].between(0, 10, inclusive='neither'), 'decil'] = '10'
result.loc[result['porc'].between(10, 20, inclusive='both'), 'decil'] = '9'
result.loc[result['porc'].between(20, 30, inclusive='neither'), 'decil'] = '8'
result.loc[result['porc'].between(30, 40, inclusive='both'), 'decil'] = '7'
result.loc[result['porc'].between(40, 50, inclusive='neither'), 'decil'] = '6'
result.loc[result['porc'].between(50, 60, inclusive='both'), 'decil'] = '5'
result.loc[result['porc'].between(60, 70, inclusive='neither'), 'decil'] = '4'
result.loc[result['porc'].between(70, 80, inclusive='both'), 'decil'] = '3'
result.loc[result['porc'].between(80, 90, inclusive='neither'), 'decil'] = '2'
result.loc[result['porc'].between(90, 101, inclusive='both'), 'decil'] = '1'

print(result.decil.value_counts().reset_index().sort_values(by=['decil'], key=lambda x: x.astype(int)))
print(result[result.Target == 1].decil.value_counts().reset_index().sort_values(by=['decil'], key=lambda x: x.astype(int)))

a = result.groupby('decil')['Prob1'].agg(min)
print(a)


In [ ]:


# test 

a = pd.DataFrame(x_test[[target_column, 'client_id']], columns=['Target', 'client_id'])
a = a.reset_index()
b = pd.DataFrame(probabilities[:,1], columns=['Prob1'])

result = pd.concat([a, b], axis=1)

result['porc'] = result['Prob1'].rank(pct=True) * 100

result['decil'] = np.where(result.Prob1 >= 0.700034               , 1, 
                            np.where((result.Prob1 >=  0.550300) & (result.Prob1 <  0.700034       ), 2,
                            np.where((result.Prob1 >=  0.426470) & (result.Prob1 < 0.550300        ) , 3,
                            np.where((result.Prob1 >=  0.317595) & (result.Prob1 < 0.426470        ), 4,
                            np.where((result.Prob1 >=  0.225329) & (result.Prob1 < 0.317595        ), 5,
                            np.where((result.Prob1 >=  0.161523) & (result.Prob1 < 0.225329        ), 6,
                            np.where((result.Prob1 >=  0.084102) & (result.Prob1 < 0.161523        ) , 7,
                            np.where((result.Prob1 >=  0.033897) & (result.Prob1 < 0.084102        ), 8,
                            np.where((result.Prob1 >=  0.020301) & (result.Prob1 < 0.033897        ), 9,
                            10)))))))))

print(result.decil.value_counts().reset_index().sort_values(by=['decil'], key=lambda x: x.astype(int)))

print(result[result.Target == 1].decil.value_counts().reset_index().sort_values(by=['decil'], key=lambda x: x.astype(int)))


# ROC curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

probabilities_train2 = clf_final_train.predict_proba(x_train[top_variables])[:, 1]
y_train = x_train[target_column].values

fpr_train, tpr_train, thresholds_train = roc_curve(x_train[target_column], probabilities_train2)
roc_auc_train = auc(fpr_train, tpr_train)

# Calculate ROC curve for training set
fpr_train, tpr_train, thresholds_train = roc_curve(y_train, probabilities_train2)
roc_auc_train = auc(fpr_train, tpr_train)

# Plot ROC curve
plt.figure(figsize=(8, 8))
plt.plot(fpr_train, tpr_train, color='darkorange', lw=2, label='Train ROC curve (area = {:.2f})'.format(roc_auc_train))
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()